In [1]:
"""
Load a BehaviourStore setup json and interactively edit it.
"""
#fom BluenetTestSuite import ...
from ipywidgets import Layout, FloatRangeSlider, Accordion, VBox, HBox, BoundedIntText, Button, ToggleButtons, FloatText
from IPython.display import display, HTML

def toggle_visibility(widget):
        isVisible = widget.layout.visibility != 'hidden'
        widget.layout.visibility ='hidden' if isVisible else 'visible'

In [50]:
def MetaDataSummary():
    indexfield = BoundedIntText(
        value=0,
        min=0,
        max=49,
        step=1,
        tooltip="index in behaviourstore",
        disabled=False,
        layout=Layout(width='100%')
    )
    
    return [indexfield]

def MetaDataDetails():
    return []

def BehaviourOverviewSummary():
    """
    Returns list of widgets for single line description of a behaviour
    """
    fromuntilfield = FloatRangeSlider(
        value=[9*60*60, 18*60*60],
        min=0,
        max=24*60*60,
        step=0.1,
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=False,
        readout_format='.1f',
        layout=Layout(width='100%')
    )
    return [fromuntilfield]

def BehaviourOverviewDetails():
    """
    Returns list of widgets for multi line description of a behaviour
    """
    typefield = ToggleButtons(
        options=['Switch', 'ExtendedSwitch', 'Twilight'],
        description='Type:',
        disabled=False,
        button_style='info',
        layout=Layout(width='100%')
    )
    
    fromfield = FloatText(
        value=9*60*60,
        description='From:',
        disabled=False
    )
    untilfield = FloatText(
        value=18*60*60,
        description='Until:',
        disabled=False
    )
    
    return [typefield, fromfield, untilfield]

def ToolbarSummary():
    editbutton = Button(
        tooltip='Edit',
        disabled=False,
        icon='fa-pencil',
        layout=Layout(width='100%')
    )
    
    return [editbutton]
    
def ToolbarDetails():
    deletebutton = Button(
        tooltip='Delete',
        disabled=False,
        icon='fa-trash-o',
        layout=Layout(width='100%')
    )
    
    savebutton = Button(
        tooltip='Save',
        disabled=False,
        icon='fa-floppy-o',
        layout=Layout(width='100%')
    )
    
    return [savebutton, deletebutton]

# def BehaviourOverview(width_str):
#     summary = BehaviourOverviewSummary()
#     details = BehaviourOverviewDetails()
    
#     return VBox(summary + details, layout=Layout(width=width_str))

# def Toolbar(width_str):
#     details = ToolbarDetails()
    
#     def toggle_toolbar_detail_widgets(b):
#         for detail_widg in details:
#             toggle_visibility(detail_widg)
        
#     summary = ToolbarSummary()
#     editbutton = summary[0]
#     editbutton.on_click(toggle_toolbar_detail_widgets)
    
#     return VBox(summary + details, layout=Layout(width=width_str, margin='5px'))

    
# def MetaData(width_str):
#     summary = MetaDataSummary()
#     details = MetaDataDetails()
    
#     return VBox(summary + details, layout=Layout(width=width_str))
    

def MakeHBox(left, left_width, middle, middle_width, right, right_width):
    hbox_layout = Layout(
        overflow='scroll',
#           border='1px solid black',
        height='',
        margin='15px 15px 0 0',
        flex_flow='row',
        display='stretch')
    
    marge = '5px'
    columns = [
        VBox(children = left, layout = Layout(width = left_width, margin=marge)),
        VBox(children = middle, layout = Layout(width = middle_width, margin=marge)),
        VBox(children = right, layout = Layout(width = right_width, margin=marge))
    ]
    
    return HBox(children = columns, layout = hbox_layout)
    
def BehaviourEntryEditor():
    """
    Returns HBox for Behaviour with additional meta-operations
    """
    summary_meta = MetaDataSummary()
    summary_overview = BehaviourOverviewSummary()
    summary_toolbar = ToolbarSummary()
    
    details_meta = MetaDataDetails()
    details_overview = BehaviourOverviewDetails()
    details_toolbar = ToolbarDetails()
    
    summary = MakeHBox(summary_meta, '5%', summary_overview, '90%' summary_toolbar, '5%')
    details = MakeHBox(details_meta, '5%', details_overview, '90%' details_toolbar, '5%')
    
    def toggle_detail_widgets(b):
        toggle_visibility(details)
            
    editbutton = summary_toolbar[0]
    editbutton.on_click(toggle_detail_widgets)
    
    
    return VBox([summary, details])

SyntaxError: invalid syntax (<ipython-input-50-b88d57921508>, line 144)

In [51]:
VBox([BehaviourEntryEditor() for i in range(3)])